In [15]:
import whisper
from pydub import AudioSegment
import os
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
import torch
import os
from __future__ import unicode_literals
import os
from GoogleImageScraper import GoogleImageScraper
from patch import webdriver_executable


url = "https://www.youtube.com/watch?v=xBMXSKXIlXs&t=429s"

# delete the audio file if it exists
if os.path.exists("audio.mp3"):
    os.remove("audio.mp3")
# use youtube_dl to download the video
os.system("yt-dlp -f 140 -o audio.mp3 " + url)

# wait until the audio file is downloaded
while not os.path.exists("audio.mp3"):
    continue

model = whisper.load_model("base")

if os.path.exists("segments"):
    os.system("rm -rf segments")

audio = AudioSegment.from_file("audio.mp3")
segment_length = 30 * 1000

if not os.path.exists("segments"):
    os.makedirs("segments")

for i, segment in enumerate(audio[::segment_length]):
    segment.export(f"segments/{i}.mp3", format="mp3")

orginal_text = ""
audio_list = os.listdir("segments")
headings = []
orginal_texts = []
dataForWeb = {
    
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = T5ForConditionalGeneration.from_pretrained("Michau/t5-base-en-generate-headline")
tokenizer = T5Tokenizer.from_pretrained("Michau/t5-base-en-generate-headline")
models = models.to(device)


[youtube] xBMXSKXIlXs: Downloading webpage
[youtube] xBMXSKXIlXs: Downloading android player API JSON
[info] xBMXSKXIlXs: Downloading 1 format(s): 140
[download] Destination: audio.mp3
[download] 100% of 8.59MiB in 00:00                      
[FixupM4a] Correcting container of "audio.mp3"


In [16]:
# storing output in json file


for i  in range(len(audio_list)):
   audio = whisper.load_audio(f"segments/{i}.mp3")
   audio = whisper.pad_or_trim(audio)
   mel = whisper.log_mel_spectrogram(audio).to(model.device)
   _, probs = model.detect_language(mel)
   options = whisper.DecodingOptions(fp16 = False)
   result = whisper.decode(model, mel, options)
   

#    building the headline
   text =  "headline: " + result.text
   max_len = 256
   encoding = tokenizer.encode_plus(text, return_tensors = "pt")
   input_ids = encoding["input_ids"].to(device)
   attention_masks = encoding["attention_mask"].to(device)
   beam_outputs = models.generate(
       input_ids = input_ids,
       attention_mask = attention_masks,
       max_length = 64,
       num_beams = 3,
       early_stopping = True,
   )
   results = tokenizer.decode(beam_outputs[0])
   headings.append(results) 
   dataForWeb[i] = {
        "heading" : results,
        "text" : result.text
    }
   
   orginal_text += "\n"
   orginal_text += "<h3>"+results + "</h3>"
   # new line
   orginal_text += "\n"
   orginal_text += "<p>"+result.text+ "</p>"

with open("text.txt", "w") as f:
    f.write(orginal_text)

In [17]:
print(dataForWeb)

{0: {'heading': '<pad> Goviy HR - What Is It All About?</s>', 'text': "Hello all, I'm Guy 3 and I'm leading the HR team at Goviy over here. So to talk about my experience in Goviy to sum it up in one word, I would definitely say that it's fulfilling. And one thing I noticed here at Goviy is if you come here, work hard, be committed, the"}, 1: {'heading': '<pad> The Learning Never Stops at Govi</s>', 'text': "And one element in Govi which I have discovered till date is that the learning never stops at Govi. And also Govi gives you a good platform to explore and try out things and which I am really happy that you know each one is given this opportunity. And I would definitely say that Govi's environment is positive, friendly."}, 2: {'heading': "<pad> Govie's 9th Year</s>", 'text': "series that is Ed to come and I wish success but nothing to Govie. Thank you so much. Hi, I'm Varsanth, Business Unit Headed Govie. Today Govie is stepping into its 9th year. My heartful wishes to Govie and ou